In [1]:
#coding:utf-8
import urllib2
from bs4 import BeautifulSoup
import sys
import re
import time
import sys

In [3]:
url=['http://news.baidu.com/n?cmd=4&class=mil&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=finannews&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=internet&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=housenews&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=autonews&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=sportnews&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=enternews&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=gamenews&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=edunews&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=healthnews&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=technnews&pn=1&from=tab'\
     ,'http://news.baidu.com/n?cmd=4&class=socianews&pn=1&from=tab']

In [5]:
ff=['E:/baidu/军事.txt'\
    ,'E:/baidu/财经.txt'\
    ,'E:/baidu/互联网.txt'\
    ,'E:/baidu/房产.txt'\
    ,'E:/baidu/汽车.txt'\
    ,'E:/baidu/体育.txt'\
    ,'E:/baidu/娱乐.txt'\
    ,'E:/baidu/游戏.txt'\
    ,'E:/baidu/教育.txt'\
    ,'E:/baidu/女人.txt'\
    ,'E:/baidu/科技.txt'\
    ,'E:/baidu/社会.txt']

In [10]:
for j in range(0,8):
    soup = BeautifulSoup(urllib2.urlopen(url[j]).read())
    main = soup.find('div',{'class':'p2'})
    index = main.find_all('a')
    len_0 = len(index)
    a = []
    for i in range(len_0):
        a.append(index[i]['href'])

    for i in range(len_0):
        try:
            soup = BeautifulSoup(urllib2.urlopen(a[i]).read())
            txt = soup.find(text=re.compile(ur"[\u4e00-\u9fa5]+"))
            txt_ = ''.join(txt)
            f = open(ff[j],'a')
            print >>f,txt_
            f.close()
        except:
            continue

In [ ]:
import jieba
import os
import sys
import codecs
from sklearn import feature_extraction
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import tree
from sklearn.naive_bayes import MultinomialNB

In [ ]:
###
def load_data():
    corpus_train = []
    target_train = []
    filepath = ''
    filelist = os.listdir(filepath)
    for num in range(len(filelist)):
        filetext = filepath+"/"+filelist[num]
        filename = os.path.basename(filetext)
        myfile = open(filetext,'r','utf-8')
        temp = myfile.readlines()
        myfile.close()
        seg_list = jieba.cut(','.join(temp[1:]),cut_all=False)
        words = " ".join(seg_list)
        target_train.append(filename)
        corpus_train.append(words)
     #-------#
    corpus_test = []
    target_test = []
    filepath = ''
    filelist = os.listdir(filepath)
    for num in range(len(filelist)):
        filetext = filepath+"/"+filelist[num]
        myfile = open(filetext,'r')
        temp = myfile.readlines()
        myfile.close()
        seg_list = jieba.cut(','.join(temp[1:]),cut_all=False)
        words = " ".join(seg_list)
        target_train.append(temp[0])
        corpus_test.append(words)
    return [[corpus_train,target_train],[corpus_test,target_test]]

def data_pro():
    [[corpus_train, target_train], [corpus_test, target_test]] = load_data()
    count_v1 = CountVectorizer()
    #该类会讲文本的词语转化为词频矩阵，矩阵元素啊a[i][j]表示j词在i类文本下的词频
    counts_train = count_v1.fit_transform(corpus_train)

    transformer = TfidfVectorizer()
    tfidf_train = transformer.fit(counts_train).transform(counts_train)

    weight_train = tfidf_train.toarray()
    count_v2 = CountVectorizer(vocabulary=count_v1.vocabulary_)
    # 让两个CountVectorizer共享vocabulary

    counts_test = count_v2.fit_transform(corpus_test)
    # fit_transform是将文本转为词频矩阵

    transformer = TfidfVectorizer()
    # 该类会统计每个词语的tf-idf权值

    tfidf_test = transformer.fit(counts_train).transform(counts_test)
    # fit_transform是计算tf-idf

    weight_test = tfidf_test.toarray()
    # weight[i][j],第i个文本，第j个词的tf-idf值
    return [[weight_train, target_train], [weight_test, target_test]]
        
[[weight_train, target_train], [weight_test, target_test]] = data_pro()

# ---------------------------------------------#
knnclf = KNeighborsClassifier()
knnclf.fit(weight_train, target_train)
knn_pred = knnclf.predict(weight_test)
# knn模型
# ---------------------------------------------#



# ---------------------------------------------#
# svm模型
svc = svm.SVC(kernel='linear')
svc.fit(weight_train, target_train)
svc_pred = svc.predict(weight_test)
# ---------------------------------------------#



# ---------------------------------------------#
# tree模型
tre = tree.DecisionTreeClassifier()
tre.fit(weight_train, target_train)
tre_pred = tre.predict(weight_test)
# ---------------------------------------------#



# ---------------------------------------------#
# bayes模型
bayes = MultinomialNB(alpha=0.01)
bayes.fit(weight_train, target_train)
bayes_pred = bayes.predict(weight_test)
# ---------------------------------------------#